## Load the preprocessed datasets
Need the preprocessed data, the id_to_word dictionaries and word_to_id dictionaries for both source and target languages.

In [10]:
from utils import load_obj

word2id_s = load_obj("DATA/preprocess/word2id_s")
word2id_t = load_obj("DATA/preprocess/word2id_t")
id2word_s = load_obj("DATA/preprocess/id2word_s")
id2word_t = load_obj("DATA/preprocess/id2word_t")

text_source = load_obj("DATA/preprocess/source_train") # Preprocessed already
text_target = load_obj("DATA/preprocess/target_train") # Preprocessed already

#raw_source_test = load_obj("DATA/preprocess/raw_source_test")
#raw_source_test = [seq.split(" ") for seq in raw_source_test]

#raw_target_test = load_obj("DATA/preprocess/raw_target_test")
#raw_target_test = [seq.split(" ") for seq in raw_target_test]
"""

def format_text(text):
    text = text.replace(" &apos;", "'")
    return text

#raw_source_test = eval("test_"+lang_s).read()
#raw_target_test = eval("test_"+lang_t).read()
print len(raw_source_test.split("\n")), len(raw_target_test.split("\n"))

"""

source_train, source_test = text_source[0:10000], text_source[10000:500]
target_train, target_test = text_target[0:10000], text_target[10000:500]
print "Vocab size is {}.".format(len(word2id_s.keys()))

Vocab size is 6003.


In [31]:
#from utils import ids_to_phrases
print len(word2id_s.keys())
vocab_size_t = len(word2id_t.keys())
vocab_size_s = len(word2id_s.keys())
lang_s = 'fr'
lang_t = 'en'
print ids_to_phrases(target_train[60], id2word_t)

print ids_to_phrases(source_train[60], id2word_s)

6003
this had to be <UNK> since , before the <UNK> citizenship legislation , a married woman was usually seen to share the nationality of her husband . 
par conséquent , <UNK> fut également l&apos; année au cours de laquelle il <UNK> possible d &quot; être citoyen canadien , en vertu de la nouvelle loi sur la citoyenneté canadienne . 


## Evaluation metric:     BLEU Score

Here I use the NLTK implementation of BLEU score to measure how successful the machine translation was.

In [ ]:
from nltk.translate import bleu_score
from utils import remove_EOS_PAD

p_phrase1 = [4,5,4,5,4,5, 1, 0]
t_phrase = [4,5,6,34,8,76, 87, 1]
# Truncate sequences at [1]
t_phrase = remove_EOS_PAD(t_phrase)
p_phrase1 = remove_EOS_PAD(p_phrase1)

print "BLEU1 score test is {}.".format(
    bleu_score.corpus_bleu([[t_phrase]], [p_phrase1], weights=([1])))
print "BLEU2 score test is {}.".format(
    bleu_score.corpus_bleu([[t_phrase]], [p_phrase1], weights=([0.5,0.5])))

## Benchmark model

Here I use the [GoogleTrans]() python package to translate the corpus by translating each individual word in the text. There seems to be an issue with the number of JSON requests the model makes to the Google Translate service, so it needs to work in passes. If a phrase is skipped due to a JSON error, keep it stored and try again later (with a new Translator instance).

In [75]:
from googletrans import Translator


def googletrans_pass(s_text, target_lang='fr'):
    translator = Translator()
    trans_corpus = []
    skipped_phrases = []
    for i, phrase in s_text:
        try:
            trans_corpus.append((i, [trans.text for 
                            trans in translator.translate(phrase, dest=target_lang)]))
        except ValueError as err:
            # Making a new Translator instance seems to help JSON errors
            translator = Translator()
            skipped_phrases.append((i, phrase))
            print "{} for phrase {}".format(err, i)
    return trans_corpus, skipped_phrases

def translate_word_by_word(source_text):
    # Need to keep track of phrase ordering by labelling
    source_text = [(i, phr) for i, phr in enumerate(source_text)]
    trans_text, skipped_phrases = googletrans_pass(source_text, target_lang=lang_t)
    print "There are {} phrases which could not be translated first time around.".format(
                                                                    len(skipped_phrases))
    # Keep making passes until there are no more untranslated phrases left
    j = 2
    while len(skipped_phrases)>0:
        translated_corpus = []
        tc_s, skipped_phrases = googletrans_pass(skipped_phrases, target_lang=lang_t)
        trans_text += tc_s
        print "There are {} phrases which could not be translated in pass {}.".format(
                                                            len(skipped_phrases), j)
        j+=1
    # Sort the phrases via their labels.
    # Sorted function gives ([indices], [phrases]) so just need 2nd element
    trans_text = zip(*sorted(zip(*BM_translated)))[1]
    return trans_text

BM_translated = translate_word_by_word(raw_source_test)
print "Corpus translated from {} to {}".format(lang_s, lang_t)


No JSON object could be decoded for phrase 46
No JSON object could be decoded for phrase 87
No JSON object could be decoded for phrase 132
No JSON object could be decoded for phrase 166
No JSON object could be decoded for phrase 217
No JSON object could be decoded for phrase 263
No JSON object could be decoded for phrase 316
No JSON object could be decoded for phrase 373
No JSON object could be decoded for phrase 430
No JSON object could be decoded for phrase 479
No JSON object could be decoded for phrase 526
No JSON object could be decoded for phrase 581
No JSON object could be decoded for phrase 630
No JSON object could be decoded for phrase 683
No JSON object could be decoded for phrase 742
No JSON object could be decoded for phrase 798
No JSON object could be decoded for phrase 1025
No JSON object could be decoded for phrase 1074
No JSON object could be decoded for phrase 1117
No JSON object could be decoded for phrase 1329
No JSON object could be decoded for phrase 1519
No JSON ob

KeyboardInterrupt: 

In [74]:
print " ".join(raw_source_test[60])
print " ".join(BM_translated[60])
print " ".join(raw_target_test[60])

Une des raisons de l&apos; échec de &quot; The Hunting Party &quot; est que l&apos; on ne laisse simplement pas Simon Hunt être le cynique déserté doté d&apos; un cœur , comme il nous est présenté au début .
A of the reasons from l&apos; failure from &quot; The Hunting Party &quot; East what l&apos; on born leash simply not Simon Hunt be the cynical deserted with d&apos; a heart , as the we East present the beginning .
One of the reasons why &quot; The Hunting Party &quot; fails is that Simon Hunt isn &apos;t allowed to simply be the run-down cynic with a heart of gold we &apos;re introduced to at the beginning .


In [ ]:
test_targets = [[ids_to_phrases(phrase,id_to_word_t)]for phrase in test_t]

BM_BLEU4 = bleu_score.corpus_bleu(test_targets, BM_translated, weights=(0.25,0.25, 0.25,0.25))
BM_BLEU2 = bleu_score.corpus_bleu(test_targets, BM_translated, weights=(0.5,0.5))
BM_BLEU1 = bleu_score.corpus_bleu(test_targets, BM_translated, weights=([1]))

print "Actual: \n", (" ".join(test_targets[10])).encode('utf-8')
print "Prediction: \n", (" ".join(BM_translated_corp[10])).encode('utf-8')
print "Unigram BLEU score is {}.".format(BM_BLEU1)
print "Bigram BLEU score is {}.".format(BM_BLEU2)
print "4-Gram BLEU: {}".format(BM_BLEU4)

# Word IDs and word-to-vec vectors

Since we are interested in the process of learning weights within the RNNs to predict seq2seq mappings rather than embeddings I have chosen to use pretrained word embeddings ino order to cut down training time. [THis is](https://sites.google.com/site/rmyeid/projects/polyglot#TOC-Download-the-Embeddings) where the embeddings are from.

In [32]:
import pickle
import numpy as np
def get_embeddings(id_to_word, lang):
    # We load pretrained word2vec embeddings from polyglot to save on training time
    filename ='DATA/polyglot-'+lang+'.pkl'
    pretrain_vocab, pretrain_embed = pickle.load(open(filename, 'rb'))
    embed_vocab = [pretrain_embed[pretrain_vocab.index('<PAD>')], pretrain_embed[pretrain_vocab.index('</S>')]]
    skip_count = 0
    skipped_words = []
    for idx, word in sorted(id_to_word.items()[2::]):
        try:
            pretrain_idx = pretrain_vocab.index(word)
            embed_vocab.append(pretrain_embed[pretrain_idx])
        except ValueError:
            try:
                # it could be that the word is a name which needs to 
                # be capitalized. Try this...
                pretrain_idx = pretrain_vocab.index(str(word.title()))
                embed_vocab.append(pretrain_embed[pretrain_idx])
            except ValueError:
                try:
                    # it could be that the word is an achronym which needs to 
                    # be upper case. Try this...
                    pretrain_idx = pretrain_vocab.index(word.upper())
                    embed_vocab.append(pretrain_embed[pretrain_idx])
                except ValueError:
                    # Give up trying to find an embedding.
                    # How many words are skipped? Which ones?
                    skip_count +=1
                    skipped_words.append(word)
                    # Let's just initialise the embedding to a random normal distribution
                    embed_vocab.append(np.random.normal(loc=0.0, scale=np.sqrt(2)/4, size=64))
    embed_vocab = np.array(embed_vocab, dtype=np.float32)
    print "The embedding matrix for {} has {} columns and {} rows.".format(lang, 
                                                embed_vocab.shape[0], embed_vocab.shape[1])
    print "{} vocab words were not in the {} embeddings file.".format(skip_count, lang)
    return embed_vocab, skipped_words
# the ith word in words corresponds to the ith embedding 

embed_vocab_s, skipped_s = get_embeddings(id2word_s, lang=lang_s)
embed_vocab_t, skipped_t = get_embeddings(id2word_t, lang=lang_t)
print skipped_t

The embedding matrix for fr has 6003 columns and 64 rows.
284 vocab words were not in the fr embeddings file.
The embedding matrix for en has 6003 columns and 64 rows.
201 vocab words were not in the en embeddings file.
[u'co-operation', u'90', u'18th', u'1970s', u'wasn', u'2nd', u'250', u'vice-president', u'on-line', u'cihr', u'didn', u'2', u'ngos', u'follow-up', u'2007', u'secretary-general', u'hasn', u'6', u'\xe2', u'long-term', u'34', u'&quot;', u'qu\xe9bec', u'2015', u'2011', u'2010', u'2013', u'2012', u'\u25aa', u'cross-border', u'so-called', u'1991', u'1990', u'1993', u'1992', u'1995', u'1994', u'1997', u'1996', u'1999', u'1998', u'fran\xe7ois', u'50,000', u'\u20ac', u'meps', u'1977', u'1976', u'1975', u'1973', u'1970', u'&#93;', u'1980s', u'100,000', u'&#91;', u'couldn', u'150', u'&apos;ll', u'aren', u'600', u'--', u'0', u'&apos;re', u'well-known', u'7', u'&apos;', u'large-scale', u'000', u'doesn', u'fran\xe7ais', u'20,000', u'&apos;ve', u'2.5', u'well-being', u'120', u'\u25e6'

** We can see above ** that the English words which were not in the embedding files are fairly specialist words or numerical values (which are the same in French) so hopefully they won't be too much of a problem.

In [33]:
#from utils import format_batch
test_x = [[5,2,3],[2], [4,2], [1,2]]
# it's going to go from the number of cols being the sequence length/ num of rows being batch size
# to the number of rows being the max sequence length/ num cols being batch size
# Essentially like a padding and then transpose
def format_batch(x):
    seq_lengths = [len(row) for row in x]
    n_batches = len(x)
    max_seq_length = max(seq_lengths)
    outputs = np.zeros(shape=(max_seq_length, n_batches),dtype=np.int32)
    for i in range(len(seq_lengths)):
        for j in range(seq_lengths[i]):
            outputs[j][i] = x[i][j]
    return outputs

print format_batch(test_x)
print np.array(format_batch(source_train[0:7]))

[[5 2 4 1]
 [2 0 2 2]
 [3 0 0 0]]
[[3704 4749 2175 3388 2376 3664 5774]
 [ 343 3146    2 2644 2063 4274 3447]
 [3146  446 1170 3939 5806  577 5076]
 [4274 1187 1843 1201 2894 2072 3146]
 [4979 5226 2442 4105  509 3146 3341]
 [ 450 5801 1201 4313 2043 3525 2355]
 [   2 1170  574 4105 1703 1325  610]
 [1201 5502 1091 2880 1004 2003 3409]
 [1187 1703 5746 2355 2009  225    2]
 [2633 4865 1703 1348 1703 1091 2009]
 [1707 1527 5816 3320 3013 4110 3409]
 [ 381 3146 4963 1703 4313  226 3309]
 [2622 5226 1170 3079 4274    1 3830]
 [2946 3396 3792  226 4639    0 1362]
 [3050    2 1538    1 4313    0 4939]
 [ 143 4313 2410    0 3674    0 3146]
 [1928 4276 1091    0 3487    0 2009]
 [ 226 1503  138    0  712    0 3396]
 [   1 3851 1654    0 2602    0 2644]
 [   0 5045 2982    0 3326    0 3938]
 [   0 2355 4082    0 4082    0 4274]
 [   0  610 2003    0 2355    0 5213]
 [   0 4865 4989    0 1091    0 2009]
 [   0 5114 4313    0 4075    0 1187]
 [   0 1201 3487    0  116    0    2]
 [   0 1170    2

Very pleasing...

# Building the network

In [34]:
import tensorflow as tf
tf.reset_default_graph()
sess = tf.InteractiveSession()
input_embedding_size = 64 # Fixed due to pretrained embedding files
encoder_hidden_units = 256
decoder_hidden_units = encoder_hidden_units # Must be the same at the moment

In [35]:
encoder_inputs = tf.placeholder(shape=(None, None), dtype=tf.int32, name='encoder_inputs')
decoder_targets = tf.placeholder(shape=(None, None), dtype=tf.int32, name='decoder_targets')
decoder_inputs = tf.placeholder(shape=(None, None), dtype=tf.int32, name='decoder_inputs')
print encoder_inputs.shape

(?, ?)


In [36]:
encoder_inputs_embedded = tf.nn.embedding_lookup(embed_vocab_s, encoder_inputs)
decoder_inputs_embedded = tf.nn.embedding_lookup(embed_vocab_t, decoder_inputs)
print encoder_inputs_embedded.shape

(?, ?, 64)


In [37]:
encoder_cell = tf.contrib.rnn.LSTMCell(encoder_hidden_units)

encoder_outputs, encoder_final_state = tf.nn.dynamic_rnn(encoder_cell, encoder_inputs_embedded,
                                                         dtype=tf.float32, time_major=True)


In [38]:
decoder_cell = tf.contrib.rnn.LSTMCell(decoder_hidden_units)

decoder_outputs, decoder_final_state = tf.nn.dynamic_rnn(
                                decoder_cell, decoder_inputs_embedded,
                                initial_state=encoder_final_state,
                                dtype=tf.float32, time_major=True, 
                                scope="plain_decoder")
print decoder_outputs

Tensor("plain_decoder/TensorArrayStack/TensorArrayGatherV3:0", shape=(?, ?, 256), dtype=float32)


In [39]:
#weights
W = tf.Variable(tf.random_uniform([decoder_hidden_units, vocab_size_t], -0.5, 0.5), dtype=tf.float32)
#bias
b = tf.Variable(tf.zeros([vocab_size_t]), dtype=tf.float32)
decoder_max_steps, decoder_batch_size, decoder_dim = tf.unstack(tf.shape(decoder_outputs))
# why do we only flatten the tensor so it's rank 2?
decoder_outputs_flat = tf.reshape(decoder_outputs, (-1, decoder_dim))
#feed flattened tensor through projection
decoder_logits_flat = tf.add(tf.matmul(decoder_outputs_flat, W), b)
# make the logits the shape of the 
decoder_logits = tf.reshape(decoder_logits_flat, (decoder_max_steps, decoder_batch_size, vocab_size_t))

print decoder_logits

Tensor("Reshape_1:0", shape=(?, ?, 6003), dtype=float32)


In [40]:
decoder_logits = tf.contrib.layers.linear(decoder_outputs, vocab_size_t)
#print decoder_logits_2
decoder_prediction = tf.argmax(decoder_logits, axis=2)
print decoder_prediction
#help(tf.argmax)

Tensor("ArgMax:0", shape=(?, ?), dtype=int64)


In [41]:
timestep_cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
    labels=tf.one_hot(decoder_targets, depth=vocab_size_t, dtype=tf.float32),
    logits=decoder_logits,
)
print timestep_cross_entropy
# loss is the mean of the cross entropy
loss = tf.reduce_mean(timestep_cross_entropy)
print loss
# We use AdaM which combines AdaGrad (parameters updated less often get updated more strongly)
# and momentum (updates depend on the slope of previous updates - avoiding local minima)
train_op = tf.train.AdamOptimizer().minimize(loss)

Tensor("Reshape_4:0", shape=(?, ?), dtype=float32)
Tensor("Mean:0", shape=(), dtype=float32)


In [42]:
sess.run(tf.global_variables_initializer())

In [43]:
# Test format_batch and make sure that the decoder
# and encoder accepts inputs with a forward pass

batch_ = [[2,124,243], [24,523,23], [9, 82]]

batch_ = format_batch(batch_)
print('batch_encoded:\n' + str(batch_))

din_ = format_batch(np.ones(shape=(3, 4), dtype=np.int32))
print('decoder inputs:\n' + str(din_))

pred_ = sess.run(decoder_prediction,
    feed_dict={
        encoder_inputs: batch_,
        decoder_inputs: din_,
    })
print('decoder predictions:\n' + str(pred_))

batch_encoded:
[[  2  24   9]
 [124 523  82]
 [243  23   0]]
decoder inputs:
[[1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]]
decoder predictions:
[[ 435 1621 3758]
 [3758  249 3758]
 [3758 3758 3758]
 [3758 3758 3758]]


In [45]:
from utils import ids_to_phrases

def batch_source_target(source, target, batch_size):
    assert len(source) == len(target)
    for start in range(0, len(source), batch_size):
        end = min(start + batch_size, len(source))
        #print type(source[start:end])
        #print len(target[start:end])
        yield source[start:end], target[start:end]     


def make_feed_dict(fd_keys, s_batch, t_batch, reverse_encoder_inputs= False):
    encoder_inputs_ = format_batch(s_batch)
    if reverse_encoder_inputs:
        encoder_inputs_ = format_batch([sequence[-2::-1]+[1] for sequence in s_batch])
    decoder_inputs_ = format_batch([[1]+sequence[0:-1] for sequence in t_batch])
    decoder_targets_ = format_batch([sequence for sequence in t_batch])
    return {
        fd_keys[0]: encoder_inputs_,
        fd_keys[1]: decoder_inputs_,
        fd_keys[2]: decoder_targets_,
    }



def make_test_feed_dict(fd_keys,s_batch, t_batch, reverse_encoder_inputs= False):
    # At testing time, we can't supervise the decoder layer with
    # the 'gold truth' example as input, so we instead feed in
    # word generated at  previous timestep. This is (apparently)
    # equivalent to feeding in zeros for the decoder inputs
    encoder_inputs_ = format_batch(s_batch)
    if reverse_encoder_inputs:
        encoder_inputs_ = format_batch([sequence[-2::-1]+[1] for sequence in s_batch])
    decoder_targets_ = format_batch([sequence for sequence in t_batch])
    decoder_inputs_ = format_batch([[0]*len(sequence) for sequence in t_batch])
    return {
        fd_keys[0]: encoder_inputs_,
        fd_keys[1]: decoder_inputs_,
        fd_keys[2]: decoder_targets_,
    }


# Test everything is working okay

batch_size = 100

for s_sample_batch, t_sample_batch in batch_source_target(source_train[0:2], target_train[0:2], batch_size):
    fd_keys = [encoder_inputs, decoder_inputs, decoder_targets]
    fd = make_feed_dict(fd_keys, s_sample_batch, t_sample_batch)
    fd_r = make_feed_dict(fd_keys, s_sample_batch, t_sample_batch, reverse_encoder_inputs= True)
    fd_t = make_test_feed_dict(fd_keys, s_sample_batch, t_sample_batch, reverse_encoder_inputs= False)
    assert len(fd.values()[0].T[0]) == len(fd_r.values()[0]) # reversed list must be the same length as original
    print fd.keys()[0]
    print ids_to_phrases(np.array(fd.values()[0]).T[0], id2word_s)
    print np.array(fd.values()[0]).T[0], "\n"
    print "Reversed as in Sutskever et al. "
    print np.array(fd_r.values()[0]).T[0]
    assert len(fd.values()[1].T[0]) == len(fd.values()[1].T[1]) # decoder inputs and targets must be the same
    
    for i in range(1, len(fd.keys())):
        print fd.keys()[i]
        ph = ids_to_phrases(np.array(fd.values()[i]).T[0], id2word_t)
        print ph
        print np.array(fd.values()[i]).T[0]
    
    print "Decoder inputs at test time"
    print np.array(fd_t.values()[1]).T[0]
    break


Tensor("encoder_inputs:0", shape=(?, ?), dtype=int32)
par conséquent , le tribunal se <UNK> pour l&apos; application du droit étranger relatif aux droits voisins . <EOS> 
[3704  343 3146 4274 4979  450    2 1201 1187 2633 1707  381 2622 2946 3050
  143 1928  226    1    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0] 

Reversed as in Sutskever et al. 
[ 226 1928  143 3050 2946 2622  381 1707 2633 1187 1201    2  450 4979 4274
 3146  343 3704    1    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0]
Tensor("decoder_inputs:0", shape=(?, ?), dtype=int32)
<EOS> by contrast , german law does not impose <UNK> <UNK> , it looks to the ultimate result . 
[   1 1041  762 3089 2091 1585 1436 1990 5021    2    2 3089 3250 5745  736
 1784 4612  213  243    0    0    0   

In [48]:
loss_track = []

In [53]:
import time

def format_idx(idx):
    # Just cuts out the padding of word index lists
    li = []
    for i in idx:
        if i ==0:
            break
        else:
            li.append(i)
    return li

BLEU = []
epochs = 30 # How many times we loop over the whole training data
batch_size = 92 # After how many sequences do we update the weights?
print "there will be {} samples in the final batch".format(len(source_train)%batch_size)
fd_keys = [encoder_inputs, decoder_inputs, decoder_targets]
try:
    batch_n = 0
    ti = time.time()
    print "training has begun..."
    for epoch in range(epochs):    
        for s_batch, t_batch in batch_source_target(source_train, target_train, batch_size):
            feed_dict = make_feed_dict(fd_keys, s_batch, t_batch)
            _, l = sess.run([train_op, loss], feed_dict)
            
            #if batch_n == 0 or batch_n == 60:
            #    
            if (batch_n==0) or (batch_n%100) == 0:
                loss_track.append(l)
                print "epoch {}".format(epoch+1)
                print 'batch {}'.format(batch_n)
                print 'loss: {}'.format(sess.run(loss, feed_dict))
                predict_ = sess.run(decoder_prediction, feed_dict)
                #predictions = [remove_EOS_PAD(pred) for pred in predict_.T]
                #actuals = [[remove_EOS_PAD(act)] for act in fd[decoder_targets].T]
                #BLEU2 = bleu_score.corpus_bleu(actuals, predictions, weights=(0.5,0.5))
                #BLEU.append(BLEU2)
                i =1
                for (inp, act, pred) in zip(feed_dict[encoder_inputs].T,
                                                         feed_dict[decoder_targets].T,
                                                         predict_.T)[10:12]:
                    print '  sample {}:'.format(i)
                    print '    input     > {} \n {}'.format(
                        format_idx(inp), ids_to_phrases(inp, id2word_s))
                    print '    actual     > {} \n {}'.format(
                        format_idx(act), ids_to_phrases(act, id2word_t).encode("utf-8"))
                    print '    predicted     > {} \n {}'.format(
                        format_idx(pred), ids_to_phrases(pred, id2word_t).encode("utf-8"))
                    i+=1
            batch_n += 1
            
    print 'Training is complete'
except KeyboardInterrupt:
    print 'training interrupted'



there will be 64 samples in the final batch
training has begun...
epoch 1
batch 0
loss: 2.13353276253
  sample 1:
    input     > [1187, 1529, 1703, 4602, 2, 3396, 1078, 1703, 2, 718, 3556, 2644, 2433, 1187, 3503, 1538, 2, 1170, 4970, 226, 1] 
 l&apos; associé de m. <UNK> s&apos; occupe de <UNK> au canada est aussi l&apos; agent qui <UNK> les fournisseurs . 
    actual     > [2077, 2, 286, 2344, 4404, 3255, 3499, 707, 5482, 3443, 2, 1022, 243, 1] 
 mr. <UNK> has a partner in canada as his agent <UNK> suppliers . 
    predicted     > [2, 2, 2, 2, 2, 2, 2] 
 <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> 
[2 2 2 2 2 2 2 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  sample 2:
    input     > [2355, 941, 2118, 3146, 4276, 4902, 1703, 1362, 1098, 2, 1410, 1098, 3108, 2, 226, 1] 
 a première vue , la profondeur de ce &quot; <UNK> politique &quot; semble <UNK> . 
    actual     > [708, 3516, 2, 3089, 1784, 2613, 5233, 1191, 10

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(len(loss_track)), loss_track)
#l = [s for i,s in sorted(zip([len(row) for row in l], l))]